In [ ]:
Train(70%), Validation(15%), & Test (15%)
Validation set actually can be regarded as a part of training set, 
because it is used to build your model, used for parameter selection.

In [ ]:
GridSearch(est, param_grid, scoring='roc_auc')

In [1]:
#一些需要的function
import numpy as np
def gauss(x,m,s):
    e=-(x-m)**2/(2*(s**2))
    g=1*np.exp(e)/(2*s*np.pi)**0.5
    return g

def remove_punc(sentences):
    import string 
    import re
    delEStr = '['+string.punctuation + u' '+']'
    delCStr = u'''[〔△〇◎◆●✦×→°☆■□◆●○๏••‧㊣℃ˍ╴←↑→．│﹙“—▲►★《》ΟΔΧ╳（）()∫&%﹪￥#@$！?,.…><*{}｢｣\\∼〈〉¬﹄•∼■`"’“×※【】  ’'　、。，‧．·‧‧：︰十；”～‘’′→＠＃＄％︿＆＊｀＝－—─＋｜／＼╲╱╱ ╱╱？『』」「\/]'''
    s=[]
    for sentence in sentences:
        if type(sentence)==unicode:
            sentence=re.sub(delEStr,u'',sentence)
            sentence=re.sub(delCStr,u'',sentence)
            s.append(sentence.encode('utf-8'))
        else:
            sentence=re.sub(delEStr,u'',sentence.decode('utf-8'))
            sentence=re.sub(delCStr,u'',sentence)
            s.append(sentence.encode('utf-8'))
    return s  


def replace_punc(sentences):
    import string 
    import re
    delEStr = '['+string.punctuation + u' '+']'
    delCStr = u'''[〔△〇◎◆●✦×→°☆■□◆●○๏••‧㊣℃ˍ╴←↑→．│﹙“—▲►★《》ΟΔΧ╳（）()∫&%﹪￥#@$！?,.…><*{}｢｣\\∼〈〉¬﹄•∼■`"’“×※【】  ’'　、。，‧．·‧‧：︰十；”～‘’′→＠＃＄％︿＆＊｀＝－—─＋｜／＼╲╱╱ ╱╱？『』」「\/]'''
    s=[]
    for sentence in sentences:
        if type(sentence)==unicode:
            sentence=re.sub(delEStr,u' ',sentence)
            sentence=re.sub(delCStr,u' ',sentence).encode('utf-8')
            sentence=sentence.split(",")
            #s.append(sentence.encode('utf-8'))
            s.append(sentence)
        else:
            sentence=re.sub(delEStr,u' ',sentence.decode('utf-8'))
            sentence=re.sub(delCStr,u' ',sentence).encode('utf-8')
            sentence=sentence.split(",")
            #s.append(sentence.encode('utf-8'))
            s.append(sentence)
    return s[0]

In [2]:
bashCommand = "rm test.txt"
import subprocess
subprocess.Popen(bashCommand.split(),cwd='/Users/jojotenya/Desktop/gensim/')

# 逐行從資料庫提出文章>斷詞
import MySQLdb as mc
import MySQLdb.cursors
import traceback
import jieba
import jieba.analyse
from gensim import corpora, models, similarities
import json
import time 

############# 基本參數設定 ###############
host='localhost'
user='root'
db="ecodaily3"
passwd=""
newstable='news2' #記得下面有個地方要改
########################################

jieba.set_dictionary('/usr/local/lib/python2.7/site-packages/jieba/dictbig.txt')
jieba.load_userdict("userdict.txt")
conn=mc.connect(host='localhost',user='root',passwd='',db=db,charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
cur=conn.cursor()
sql='select content from news2'
cur.execute(sql)
f=open('/Users/jojotenya/Desktop/gensim/test.txt','a')
texts=cur.fetchone()
s=open('/Users/jojotenya/Desktop/gensim/stopword.txt',"r")
stoplist=[]
for stop in s.readlines():
    stoplist.append(stop.strip())
s.close()
stoplist=set(stoplist)
while texts is not None:
    words = jieba.cut(replace_punc([texts[0]])[0], cut_all=False)
    sentence=[]
    for w in words:
        if len(w.strip())<=1:
            continue
        elif w.encode('utf-8') in stoplist:
            #print "stop: ", w
            continue
        else:
            #print "enter: ",w
            sentence.append(w)
    feed=(' '.join(sentence)).encode('utf-8')
    f.write(feed)
    f.write("\n")
    texts=cur.fetchone()

cur.close()
conn.close()
f.close()


#轉換斷詞為純數字vector，生成dictionary詞庫、詞頻統計
#stoplist=[]
dictionary = corpora.Dictionary(line.split() for line in open('/Users/jojotenya/Desktop/gensim/test.txt'))
#stop_ids=[]
#for stopword in stoplist:
#    if stopword.decode('utf-8') in dictionary.token2id:
#        stop_ids.append(stopword.decode('utf-8'))
#once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.iteritems() if docfreq == 1]
#once_ids = []
#dictionary.filter_tokens(stop_ids + once_ids) # remove stop words and words that appear only once
#dictionary.filter_tokens(set(stop_ids))
#dictionary.compactify()
dictionary.save('/Users/jojotenya/Desktop/gensim/test.dict')

class MyCorpus(object):
    def __iter__(self):
        for line in open('/Users/jojotenya/Desktop/gensim/test.txt'):
            yield dictionary.doc2bow(line.split())
corpus = MyCorpus() 
corpora.MmCorpus.serialize('/Users/jojotenya/Desktop/gensim/test.mm', corpus)

'''for c in corpus_to_use:
    print json.dumps(c,ensure_ascii=False)'''

#做tfidf，將斷詞編號依照vector轉換回原本的詞，一同將詞與權重存入database
#Lsi降維，將各topic與權重存入database
#print json.dumps(dictionary.token2id, encoding="utf-8", ensure_ascii=False)
##no update：
#dictionary=corpora.Dictionary.load('/Users/jojotenya/Desktop/test.dict')
#corpus=corpora.MmCorpus('/Users/jojotenya/Desktop/test.dict')
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
t=[]
for doc in corpus_tfidf:
     t.append([[dictionary.token2id.keys()[dictionary.token2id.values().index(d[0])],d[1]] for d in doc])      

c=[]
for doc in corpus:
     c.append([d[1] for d in doc])

for i in xrange(0,60):
    for j in xrange(0,len(t[i])):
        t[i][j].append(c[i][j])     
        
try:
    conn=mc.connect(host='localhost',user='root',passwd='',db=db,charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
    cur=conn.cursor()
    sql='select newsID from news2'
    cur.execute(sql)
    row=cur.fetchone()
    conn2=mc.connect(host='localhost',user='root',passwd='',db=db,charset="utf8")
    conn2.query('SET autocommit=TRUE;')
    cur2=conn2.cursor()
    #print json.dumps(t[i],ensure_ascii=False)
    i=0
    while row is not None:    
        for s in t[i]:  
            data=(int(row[0]),s[0],s[1],s[2])
            cur2.execute('insert into keywords(keyid,term,weight,freq) values(%s,%s,%s,%s)',data)
            #conn2.commit()
        for c in l[i]: 
            data=(int(row[0]),c[0],c[1])
            cur3.execute('insert into lsi(lsiid,topic,weight) values(%s,%s,%s)',data)
            #conn3.commit()
        i+=1
        row=cur.fetchone()
except:
    print traceback.format_exc()

finally:
    cur.close()
    conn.close()
    cur2.close()
    conn2.close()

Building prefix dict from /usr/local/lib/python2.7/site-packages/jieba/dictbig.txt ...
DEBUG:jieba:Building prefix dict from /usr/local/lib/python2.7/site-packages/jieba/dictbig.txt ...
Loading model from cache /var/folders/pn/qc_lny8n1cv6pt_2tqlg4s6m0000gn/T/jieba.u91a3eae8b085aaf1cd1517c2fdfc2047.cache
DEBUG:jieba:Loading model from cache /var/folders/pn/qc_lny8n1cv6pt_2tqlg4s6m0000gn/T/jieba.u91a3eae8b085aaf1cd1517c2fdfc2047.cache
Loading model cost 1.023 seconds.
DEBUG:jieba:Loading model cost 1.023 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


Traceback (most recent call last):
  File "<ipython-input-2-9c9b55f749b4>", line 117, in <module>
    cur2.execute('insert into keywords(keyid,term,weight,freq) values(%s,%s,%s,%s)',data)
  File "/usr/local/lib/python2.7/site-packages/MySQLdb/cursors.py", line 205, in execute
    self.errorhandler(self, exc, value)
  File "/usr/local/lib/python2.7/site-packages/MySQLdb/connections.py", line 36, in defaulterrorhandler
    raise errorclass, errorvalue
IntegrityError: (1062, "Duplicate entry '1-\xe7\x8f\xbe\xe4\xbb\xbb' for key 'PRIMARY'")



In [3]:
### CV to find best lsi - GaussianNB

In [51]:
############# 基本參數設定 ###############
host='localhost'
user='root'
db="ecodaily3"
passwd=""
newstable='news2'  #記得下面有個地方要改
rang1=5
rang2=26
#num_topics=10
########################################
meanstd=[]
for num_topics in xrange(rang1,rang2):
    #lsi
    a= time.clock()

    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=num_topics)
    lsi.save('/Users/jojotenya/Desktop/gensim/test.lsi')
    corpus_lsi = lsi[corpus_tfidf]
    l=[]
    for c_lsi in corpus_lsi:
        l.append(c_lsi)  
    try:
        conn=mc.connect(host=host,user=user,passwd=passwd,db=db,charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
        cur=conn.cursor()
        sql_drop_lsi="drop table if exists lsi;"
        cur.execute(sql_drop_lsi)
        conn.commit()
        sql_create_lsi='''
        create table lsi(
        lsiid int not null,
        topic int not null,
        weight float,
        primary key pk_lsi (lsiid,topic)
        );
        '''
        cur.execute(sql_create_lsi)
        conn.commit()

        sql_drop_lsiStatistic="drop table if exists lsiStatistic;"
        cur.execute(sql_drop_lsiStatistic)
        conn.commit()
        sql_create_lsiStatistic='''
            create table lsiStatistic(
            `class` varchar(10),
            topic int not null,
            mean float,
            std float,
            primary key pk_lsiStatistic (`class`,topic)
            );
        '''
        cur.execute(sql_create_lsiStatistic)
        conn.commit()

        sql='select newsID from %s'%newstable
        cur.execute(sql)
        row=cur.fetchone()
        conn3=mc.connect(host=host,user=user,passwd=passwd,db=db,charset="utf8")
        conn3.query('SET autocommit=TRUE;')
        cur3=conn3.cursor()
        i=0
        while row is not None:    
            for c in l[i]: 
                data=(int(row[0]),c[0],c[1])
                cur3.execute('insert into lsi(lsiid,topic,weight) values(%s,%s,%s)',data)
                #conn3.commit()
            i+=1
            row=cur.fetchone()
    except:
        print traceback.format_exc()

    finally:
        cur.close()
        conn.close()
        cur3.close()
        conn3.close()

    #每個分類的lsi-mu,lsi-sigma
    conn=mc.connect(host='localhost',user='root',passwd='',
                    db=db,charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
    cur=conn.cursor()
    cur.execute('select distinct(`class`) from %s'%newstable)
    categories=cur.fetchall()

    conn2=mc.connect(host='localhost',user='root',passwd='',
                    db=db,charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
    conn2.query('SET autocommit=TRUE;')
    cur2=conn2.cursor()

    for i in xrange(0,len(categories)):
        cur.execute('select topic,AVG(weight),STD(weight) from lsi where lsiid in \
                    (select newsID from news2 where class = %s) \
                    GROUP BY topic',(categories[i][0],))
        result=cur.fetchone()
        while result is not None:
            data=(categories[i],result[0],result[1],result[2])
            cur2.execute('insert into lsiStatistic(`class`,topic,mean,std) value(%s,%s,%s,%s)',data)
            #conn2.commit()
            result=cur.fetchone()

    cur.close()
    conn.close()
    cur2.close()
    conn2.close()

    # create train set and test it
    conn=mc.connect(host=host,user=user,passwd=passwd,
                    db=db,charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
    cur=conn.cursor()
    cur.execute('select newsID,reclass from %s where reclass is not null '%newstable)
    idClass=[i for i in cur.fetchall()]
    ID=[i[0] for i in idClass]
    classSetRaw=[i[1] for i in idClass]
    categories=set(classSetRaw)
    classdic={}
    for i,j in enumerate(categories):
        classdic[j]=i
    classSet=[classdic[c] for c in classSetRaw]     #for train and test
    dataSet=[]
    for i in ID:
        cur.execute('select weight from lsi where lsiid = %s'%i)
        ds=[i[0] for i in cur.fetchall()]
        dataSet.append(ds)

    classSet=np.array(classSet)
    dataSet=np.array(dataSet)
    cur.close()
    conn.close()

    dataTrain=[]
    dataTestSet=[]
    classTrain=[]
    classTestSet=[]
    from sklearn.cross_validation import StratifiedKFold
    skf = StratifiedKFold(classSet, 10,shuffle=False, random_state=0)  
    for train_index, test_index in skf:
        #print("TRAIN:", train_index, "TEST:", test_index)
        dataTrain.append(dataSet[train_index])
        dataTestSet.append(dataSet[test_index])
        classTrain.append(classSet[train_index])
        classTestSet.append(classSet[test_index])
    #print classTrain[0].shape
    #print dataTrain[0].shape

    '''from sklearn.cross_validation import train_test_split 
    for i in xrange(0,10):
        X=dataTrain[i]
        y=classTrain[i]
        dataTrain, dataValidation, classTrain, classValidation = train_test_split(X,y, test_size=0.22, random_state=0)  #X放資料,y放類別,type:nparray 
    #    print("train data shape: %r, train target shape: %r"
    #          % (dataTrain.shape, classTrain.shape)) # train data shape: (1437, 64), train target shape: (1437,)
    #    print("validation data shape: %r, validation target shape: %r"
    #          % (dataValidation.shape, classValidation.shape)) # test data shape: (360, 64), test target shape: (360,)'''

    from sklearn.cross_validation import train_test_split 
    dataTrainSet=[]
    dataValSet=[]
    classTrainSet=[]
    classValSet=[]
    for i in range(0,10):
        X=dataTrain[i]
        y=classTrain[i]
        dt, dv, ct, cv = train_test_split(X,y, test_size=0.22, random_state=0)  #X放資料,y放類別,type:nparray 
        dataTrainSet.append(dt)
        dataValSet.append(dv)
        classTrainSet.append(ct)
        classValSet.append(cv)
        #print("train data shape: %r, train target shape: %r"% (dt.shape, ct.shape)) # train data shape: (1437, 64), train target shape: (1437,)
        #print("validation data shape: %r, validation target shape: %r"% (dv.shape, cv.shape))
    dataTrainSet=np.array(dataTrainSet)
    dataValSet=np.array(dataValSet)
    classTrainSet=np.array(classTrainSet)
    classValSet=np.array(classValSet)

    from sklearn.naive_bayes import GaussianNB    
    import pandas as pd
    scores=[]

    for i in range(0,10):
        dic={}
        clf = GaussianNB()
        clf.fit(dataTrainSet[i], classTrainSet[i])
        train_score = clf.score(dataTrainSet[i], classTrainSet[i])
        #print train_scores
        val_score = clf.score(dataValSet[i], classValSet[i])
        #print val_scores
        test_score = clf.score(dataTestSet[i], classTestSet[i])
        #print test_scores
        #print "---"
        #f.write('\t-fold'+str(i)+': \n')
        #f.write('\ttrain_score: '+str(train_score)+'\n')
        #f.write('\tvalidation_score: '+str(val_score)+'\n')
        #f.write('\tttest_score: '+str(test_score)+'\n')
        #train_scores.append(train_score)
        #val_scores.append(val_score)
        #test_scores.append(test_score)
        dic['train_scores']=train_score
        dic['val_scores']=val_score
        dic['test_scores']=test_score
        scores.append(dic)
    df = pd.DataFrame(scores)
    #df.to_csv('/Users/jojotenya/Desktop/GaussNB/GaussNB'+'_'+str(num_topics)+'.csv',encoding="utf-8")

    
    # 算CV的mean,std
    from scipy.stats import sem
    train_scores=[dic['train_scores'] for dic in scores]
    val_scores =[dic['val_scores'] for dic in scores]
    test_scores=[dic['test_scores'] for dic in scores]
    train_scores_mean=np.mean(train_scores)
    val_scores_mean=np.mean(val_scores)
    test_scores_mean=np.mean(test_scores)
    total_mean=np.mean(test_scores+train_scores+val_scores)
    train_scores_std=sem(train_scores)
    val_scores_std=sem(val_scores)
    test_scores_std=sem(test_scores)
    total_std=sem(test_scores+train_scores+val_scores)  
    meandic={}
    meandic["num_topics"]=num_topics
    meandic["Mean(train)"]=train_scores_mean
    meandic["Mean(validation)"]=val_scores_mean
    meandic["Mean(test)"]=test_scores_mean
    meandic["Mean(total)"]=total_mean
    meandic["std(train)"]=train_scores_std
    meandic["std(validation)"]=val_scores_std
    meandic["std(test)"]=test_scores_std
    meandic["std(total)"]=total_std
    meanstd.append(meandic)
    #print '========================================='
    #print 'num_topics: ',num_topics
    #print df
    #print ("Mean score(train): {0:.3f} (+/-{1:.3f})").format(train_scores_mean,train_scores_std)
    #print ("Mean score(validation): {0:.3f} (+/-{1:.3f})").format(val_scores_mean,val_scores_std)
    #print ("Mean score(test): {0:.3f} (+/-{1:.3f})").format(test_scores_mean,test_scores_std)
    #print '------------------------------------'
    #print ("Mean score(total): {0:.3f} (+/-{1:.3f})").format(total_mean,total_std)  
    print time.clock()-a
df2=pd.DataFrame(meanstd)
df2.to_csv('/Users/jojotenya/Desktop/GaussNB/GaussNB'+'_'+str(rang1)+'_'+str(rang2-1)+'.csv',encoding="utf-8")

0.270159
0.265904
0.352915
0.389943
0.400208
0.391812
0.41543
0.420937
0.43689
0.51424
0.384048
0.386632
0.392296
0.392051
0.4114
0.483309
0.468649
0.44848
0.516124
0.485424
0.464713


In [40]:
### CV to find best lsi - SVM

In [ ]:
### CV to find best lsi - SVM -ROC -tune

In [68]:
# SVM-ROC
####################
n_gammas = 5
gammas = np.logspace(-7, -1, n_gammas)
n_Cs = 5
Cs = np.logspace(-5, 5, n_Cs)
####################

svmcsv=[]
print "train-validation:"
for num_topics in xrange(rang1,rang2):
    for l,gamma in enumerate(gammas):
        for m,C in enumerate(Cs):
            conn=mc.connect(host=host,user=user,passwd=passwd,
                            db=db,charset="utf8",cursorclass = MySQLdb.cursors.SSCursor)
            cur=conn.cursor()
            cur.execute('select newsID,reclass from %s where reclass is not null '%newstable)
            idClass=[i for i in cur.fetchall()]
            ID=[i[0] for i in idClass]
            classSetRaw=[i[1] for i in idClass]
            categories=set(classSetRaw)
            classdic={}
            for i,j in enumerate(categories):
                classdic[j]=i
            classSet=[classdic[c] for c in classSetRaw]     #for train and test
            dataSet=[]
            for i in ID:
                cur.execute('select weight from lsi where lsiid = %s'%i)
                ds=[i[0] for i in cur.fetchall()]
                dataSet.append(ds)

            classSet=np.array(classSet)
            dataSet=np.array(dataSet)
            cur.close()
            conn.close()

            dataTrain=[]
            dataTestSet=[]
            classTrain=[]
            classTestSet=[]
            skf = StratifiedKFold(classSet, 10,shuffle=False, random_state=0)  
            for train_index, test_index in skf:
                #print("TRAIN:", train_index, "TEST:", test_index)
                dataTrain.append(dataSet[train_index])
                dataTestSet.append(dataSet[test_index])
                classTrain.append(classSet[train_index])
                classTestSet.append(classSet[test_index])
            #print classTrain[0].shape
            #print dataTrain[0].shape


            dataTrainSet=[]
            dataValSet=[]
            classTrainSet=[]
            classValSet=[]
            for i in range(0,10):
                X=dataTrain[i]
                y=classTrain[i]
                dt, dv, ct, cv = train_test_split(X,y, test_size=0.22, random_state=0)  #X放資料,y放類別,type:nparray 
                dataTrainSet.append(dt)
                dataValSet.append(dv)
                classTrainSet.append(ct)
                classValSet.append(cv)
                #print("train data shape: %r, train target shape: %r"% (dt.shape, ct.shape)) # train data shape: (1437, 64), train target shape: (1437,)
                #print("validation data shape: %r, validation target shape: %r"% (dv.shape, cv.shape))
            dataTrainSet=np.array(dataTrainSet)
            dataValSet=np.array(dataValSet)
            classTrainSet=np.array(classTrainSet)
            classValSet=np.array(classValSet)

            n_classes=len(set(classSet))
            from sklearn.preprocessing import label_binarize
            classTrainSet_b=[label_binarize(d, classes=list(set(classSet))) for d in classTrainSet]
            classValSet_b=[label_binarize(d, classes=list(set(classSet))) for d in classValSet]
            classTestSet_b=[label_binarize(d, classes=list(set(classSet))) for d in classTestSet]

            #-----------------------
            from sklearn import svm, datasets
            from sklearn.metrics import roc_curve, auc
            from sklearn.cross_validation import train_test_split
            from sklearn.multiclass import OneVsRestClassifier
            X_train=dataTrainSet[0]
            y_tr=classTrainSet_b[0]
            y_train=[]
            for y in y_tr:
                if y == [0]:
                    y_train.append([1,0])
                elif y == [1]:
                    y_train.append([0,1])
                else:
                    y_train.append(y)
                    
            X_test=dataValSet[0]
            y_te=classValSet_b[0]
            y_test=[]
            for y in y_te:
                if y == [0]:
                    y_test.append([1,0])
                elif y == [1]:
                    y_test.append([0,1])
                else:
                    y_test.append(y)
            y_train=np.array(y_train)
            y_test=np.array(y_test)
            #print X_train.shape
            #print y_train.shape
            #print X_test.shape
            #print y_train
            #print y_test
            # Learn to predict each class against the other
            classifier = OneVsRestClassifier(svm.SVC(C=C,gamma=gamma,kernel='linear', probability=True,
            random_state=0))
            y_score = classifier.fit(X_train, y_train).decision_function(X_test)
            #print y_score

            # Compute ROC curve and ROC area for each class
            fpr = dict()
            tpr = dict()
            roc_auc = dict()
            for i in range(n_classes):
                fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
                roc_auc[i] = auc(fpr[i], tpr[i])

            # Compute micro-average ROC curve and ROC area
            fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
            roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

            # Plot ROC curve
            plt.figure()
            plt.plot(fpr["micro"], tpr["micro"],
                     label='micro-average ROC curve (area = {0:0.2f})'
                           ''.format(roc_auc["micro"]))
            for i in range(n_classes):
                plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                               ''.format(i, roc_auc[i]))

            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('1-SPC')
            plt.ylabel('TPR')
            plt.title('ROC:num_topics='+str(num_topics)+',gamma='+str(gamma)+',C='+str(C))
            plt.legend(loc="lower right")
            plt.savefig('/Users/jojotenya/Desktop/ROC/ROC_SVM_'+str(num_topics)+str(l)+str(m)+'.png',bbox_inches='tight')
            plt.close()
            #plt.show()
            svmpreffix={}
            svmpreffix["num_topics"]=num_topics
            svmpreffix["gamma"]=gamma
            svmpreffix["C"]=C
            svmcsv.append(dict(svmpreffix.items()+roc_auc.items()))
import pandas as pd
df=pd.DataFrame(svmcsv)
df.to_csv('/Users/jojotenya/Desktop/ROC/SVM'+'_'+str(rang1)+'_'+str(rang2-1)+'.csv',encoding="utf-8")
#print 'num_topics: ',num_topics
            
            


train-validation:


In [ ]:
### CV to find best lsi - SVM -ROC -test

In [42]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

# Import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target
# Binarize the output
y = label_binarize(y, classes=[0, 1, 2])
n_classes = y.shape[1]

# Add noisy features to make the problem harder
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
                                                    random_state=0)

'''print X_train.shape
print y_train.shape
print X_test.shape
print y_train'''
# Learn to predict each class against the other
#print y_train
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
                                 random_state=0))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)
#print y_score

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr[2], tpr[2], label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
#plt.show()

# Plot ROC curve
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                   ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

[[1 0 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [1 0 0]]
[[-0.76305896 -0.36472635  0.1239796 ]
 [-0.20238399 -0.63148982 -0.16616656]
 [ 0.11808492 -0.80262259 -0.32062486]
 [-0.90750303 -0.1239792   0.02184016]
 [-0.01108555 -0.27918155 -0.71882525]
 [-0.60521037 -0.34728075 -0.05851787]
 [ 0.02264569 -0.24507866 -0.79094201]
 [-0.61115098  0.1827259  -0.57154703]
 [-0.3759064

In [ ]:
### Grid

In [38]:
# decide gamma
n_gammas = 10
n_iter = 5
cv = ShuffleSplit(n_samples, n_iter=n_iter, train_size=500, test_size=500, random_state=0)
train_scores = np.zeros((n_gammas, n_iter))  #創造array空間 5*10
test_scores = np.zeros((n_gammas, n_iter))
gammas = np.logspace(-7, -1, n_gammas)
#create gammas array，稍微有了解過 libsvm 的人, 會知道要帶入 gamma 的差距通常會給上下界
#然後抓 log 曲線(也就是一般帶數字是 1,2,3,4 彼此差 1 他則是從 10-7帶到10-1, 彼此差 log 距離)
 
for i, gamma in enumerate(gammas):
    for j, (train, test) in enumerate(cv):
        clf = SVC(C=10, gamma=gamma).fit(X[train], y[train])
        train_scores[i, j] = clf.score(X[train], y[train])
        test_scores[i, j] = clf.score(X[test], y[test])
        
for i in range(n_iter):
    pl.semilogx(gammas, train_scores[:, i], alpha=0.4, lw=2, c='b')
    pl.semilogx(gammas, test_scores[:, i], alpha=0.4, lw=2, c='g')
pl.ylabel("score for SVC(C=10, gamma=gamma)")
pl.xlabel("gamma")
pl.text(1e-6, 0.5, "Underfitting", fontsize=16, ha='center', va='bottom')
pl.text(1e-4, 0.5, "Good", fontsize=16, ha='center', va='bottom')
pl.text(1e-2, 0.5, "Overfitting", fontsize=16, ha='center', va='bottom')
pl.show()

In [ ]:
# decide C
n_Cs = 10
n_iter = 5
cv = ShuffleSplit(n_samples, n_iter=n_iter, train_size=500, test_size=500,
    random_state=0)
 
train_scores = np.zeros((n_Cs, n_iter))
test_scores = np.zeros((n_Cs, n_iter))
Cs = np.logspace(-5, 5, n_Cs)
 
for i, C in enumerate(Cs):
    for j, (train, test) in enumerate(cv):
        clf = SVC(C=C, gamma=1e-3).fit(X[train], y[train])
        train_scores[i, j] = clf.score(X[train], y[train])
        test_scores[i, j] = clf.score(X[test], y[test])
        
for i in range(n_iter):
    pl.semilogx(Cs, train_scores[:, i], alpha=0.4, lw=2, c='b')
    pl.semilogx(Cs, test_scores[:, i], alpha=0.4, lw=2, c='g')
pl.ylabel("score for SVC(C=C, gamma=1e-3)")
pl.xlabel("C")
pl.text(1e-3, 0.5, "Underfitting", fontsize=16, ha='center', va='bottom')
pl.text(1e3, 0.5, "Few Overfitting", fontsize=16, ha='center', va='bottom')
pl.show()

In [ ]:
### Cross Validation

In [ ]:
import time 
a=time.clock()
# 試切一份
from sklearn.cross_validation import train_test_split 
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  #X放資料,y放類別,type:nparray 
print("train data shape: %r, train target shape: %r"
      % (X_train.shape, y_train.shape)) # train data shape: (1437, 64), train target shape: (1437,)
print("test data shape: %r, test target shape: %r"
      % (X_test.shape, y_test.shape)) # test data shape: (360, 64), test target shape: (360,)
svc = SVC(kernel='rbf', C=10, gamma=0.001).fit(X_train, y_train)

# CV
from sklearn.cross_validation import ShuffleSplit
cv = ShuffleSplit(n_samples, n_iter=10, test_size=0.1,random_state=0)
for cv_index, (train, test) in enumerate(cv):
    print("# Cross Validation Iteration #%d" % cv_index)
    print("train indices: {0}...".format(train[:10]))
    print("test indices: {0}...".format(test[:10]))
    print("train score: {0:.3f}, test score: {1:.3f}\n".format(
        svc.score(X[train], y[train]), svc.score(X[test], y[test])))
'''
# 只會得到test score但速度快
from sklearn.cross_validation import cross_val_score
cv = ShuffleSplit(n_samples, n_iter=10, test_size=0.1, random_state=0)
 
test_scores = cross_val_score(svc, X, y, cv=cv, n_jobs=2,scoring='roc_auc') # use default svc
print test_scores
'''

# 算CV的mean,std
from scipy.stats import sem
def mean_score(scores):
    """Print the empirical mean score and standard error of the mean."""
    return ("Mean score: {0:.3f} (+/-{1:.3f})").format(np.mean(scores), sem(scores))
print(mean_score(test_scores)) # Mean score: 0.489 (+/-0.023)
time.clock()-a

In [28]:
### GridSearchCV

In [41]:
from sklearn.grid_search import GridSearchCV
from pprint import pprint
svc_params = {
    'C': np.logspace(-1, 2, 4),
    'gamma': np.logspace(-4, 0, 5),
}
 
n_subsamples = 500
X_small_train, y_small_train = X_train[:n_subsamples], y_train[:n_subsamples]
 
gs_svc = GridSearchCV(SVC(), svc_params, cv=10, n_jobs=-1)
 
gs_svc.fit(X_small_train, y_small_train)
 
print gs_svc.best_params_, gs_svc.best_score_ # {'C': 10.0, 'gamma': 0.001} 0.982

{'C': 1.0, 'gamma': 0.001} 0.98


In [76]:
import numpy as np
from sklearn.cross_validation import KFold
kf = KFold(4, n_folds=2)
for train, test in kf:
    print("%s %s" % (train, test))

X = np.array([[0., 0.], [1., 1.], [-1., -1.], [2., 2.]])
y = np.array([0, 1, 0, 1])
X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
print X_train
print X_test
print y[train]
print y[test]

[2 3] [0 1]
[0 1] [2 3]
[[ 0.  0.]
 [ 1.  1.]]
[[-1. -1.]
 [ 2.  2.]]
[0 1]
[0 1]


In [67]:
from sklearn.cross_validation import StratifiedKFold
labels = [0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
skf = StratifiedKFold(labels, 4)
for train, test in skf:
    print("%s %s" % (train, test))

    
X = np.array([[0., 0.], [1., 1.], [-1., -1.], [2., 2.]])
y = np.array([0, 1, 0, 1])
X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]

[1 2 3 6 7 8 9] [0 4 5]
[0 2 3 4 5 8 9] [1 6 7]
[0 1 3 4 5 6 7 9] [2 8]
[0 1 2 4 5 6 7 8] [3 9]


IndexError: index 4 is out of bounds for axis 0 with size 4